In [1]:
import PIL

In [1]:
import random

with open("mem.txt", "w") as f:
    for _ in range(209):
        f.write(str(random.randint(0, 7)) + "\n")


In [5]:
import os
import numpy as np
from PIL import Image

input_folder = "../sprites_cropped"
output_folder = "./coe/walk"

os.makedirs(output_folder, exist_ok=True)

def make_coe(file):
    image = Image.open(file).convert("RGBA")

    # Composite over black so transparent pixels = (0,0,0)
    background = Image.new("RGBA", image.size, (0, 0, 0, 255))
    image = Image.alpha_composite(background, image)
    image = image.convert("RGB")

    arr = np.array(image)
    height, width, _ = arr.shape

    # Convert to RGB444
    r = (arr[:, :, 0] >> 4).astype(np.uint16)
    g = (arr[:, :, 1] >> 4).astype(np.uint16)
    b = (arr[:, :, 2] >> 4).astype(np.uint16)
    rgb444 = (r << 8) | (g << 4) | b
    rgb444 = rgb444.flatten()

    # Format to hex
    rgb444_hex = [format(val, "03X") for val in rgb444]
    image_string = ",\n".join(rgb444_hex) + ";"

    coe_comments = (
        f";Image stored row-major\n"
        f";image_rows={height}\n"
        f";image_columns={width}\n"
        f";total_pixels={height * width}\n"
        f";total_size={height * width*12} bytes\n"
    )
    coe_header = "memory_initialization_radix=16;\nmemory_initialization_vector=\n"
    return coe_comments + coe_header + image_string

for filename in os.listdir(input_folder):
    if filename.lower().endswith(".png"):
        file_path = os.path.join(input_folder, filename)
        coe_path = os.path.join(output_folder, f"{filename[:-4]}.coe")
        with open(coe_path, "w") as f:
            f.write(make_coe(file_path))
        print(f"Generated: {coe_path}")


Generated: ./coe/walk/sprite_8.coe
Generated: ./coe/walk/sprite_0.coe
Generated: ./coe/walk/sprite_1.coe
Generated: ./coe/walk/sprite_3.coe
Generated: ./coe/walk/sprite_2.coe
Generated: ./coe/walk/sprite_6.coe
Generated: ./coe/walk/sprite_7.coe
Generated: ./coe/walk/sprite_5.coe
Generated: ./coe/walk/sprite_4.coe
